<a href="https://colab.research.google.com/github/meredith224/Pandas_DE_Academy/blob/main/Pandas_missing_values_multiple_datasets_Tyler_7_11_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

np.random.seed(42)
random.seed(42)

# ========== 1️⃣ Products Table ========== #
n_products = 100
product_ids = [f"PRD{str(i).zfill(4)}" for i in range(n_products)]
categories = ["Apparel", "Accessories", "Home", "Beauty", "Electronics", None]  # Some missing categories

products_df = pd.DataFrame({
    "ProductID": product_ids,
    "ProductName": [f"Product {i}" for i in range(n_products)],
    "Category": np.random.choice(categories, n_products, p=[0.25, 0.25, 0.2, 0.15, 0.1, 0.05]),
    "Price": np.round(np.random.uniform(5, 500, n_products), 2),
    "Description": [f"Description for product {i}" if np.random.rand() > 0.1 else None for i in range(n_products)]  # 10% missing
})

# Introduce price errors (out-of-date prices)
price_update_indexes = random.sample(range(n_products), 10)
products_df.loc[price_update_indexes, "Price"] = products_df.loc[price_update_indexes, "Price"] * 0.5

# ========== 2️⃣ Customers Table ========== #
n_customers = 200
customer_ids = [f"CUST{str(i).zfill(4)}" for i in range(n_customers)]

customers_df = pd.DataFrame({
    "CustomerID": np.random.choice(customer_ids + random.choices(customer_ids, k=20), n_customers),  # duplicates
    "Name": [f"Customer {i}" for i in range(n_customers)],
    "Email": [f"customer{i}@example.com" if np.random.rand() > 0.1 else f"bad_email_{i}" for i in range(n_customers)],  # 10% bad
    "Address": [f"{random.randint(100,999)} Main St" if np.random.rand() > 0.15 else None for i in range(n_customers)]  # 15% missing
})

# ========== 3️⃣ Orders Table ========== #
n_orders = 500
order_ids = [f"ORD{str(i).zfill(5)}" for i in range(n_orders)]
statuses = ["Paid", "Pending", "Shipped", "Cancelled", "pending", "PAID", "Refunded"]  # intentional case issues

orders_df = pd.DataFrame({
    "OrderID": order_ids,
    "CustomerID": np.random.choice(customer_ids + [None]*50, n_orders),  # guest checkouts (missing IDs)
    "ProductID": np.random.choice(product_ids, n_orders),
    "OrderDate": [
        (datetime(2022, 1, 1) + timedelta(days=random.randint(0, 730))).strftime(
            "%Y-%m-%d" if random.random() > 0.2 else "%d/%m/%Y"
        ) for _ in range(n_orders)
    ],  # Mixed date formats
    "Quantity": np.random.randint(1, 5, n_orders),
    "Status": np.random.choice(statuses, n_orders)
})


In [ ]:
print(products_df)
print(customers_df)
print(orders_df)

   ProductID ProductName     Category   Price                 Description
0    PRD0000   Product 0  Accessories   20.56   Description for product 0
1    PRD0001   Product 1         None  320.02                        None
2    PRD0002   Product 2       Beauty  160.61   Description for product 2
3    PRD0003   Product 3         Home  128.37   Description for product 3
4    PRD0004   Product 4      Apparel  454.25   Description for product 4
..       ...         ...          ...     ...                         ...
95   PRD0095  Product 95  Accessories  177.86  Description for product 95
96   PRD0096  Product 96         Home  364.35  Description for product 96
97   PRD0097  Product 97  Accessories  449.07  Description for product 97
98   PRD0098  Product 98      Apparel  444.11  Description for product 98
99   PRD0099  Product 99      Apparel  391.04                        None

[100 rows x 5 columns]
    CustomerID          Name                    Email      Address
0     CUST0108    Cus

In [ ]:
# === Investigate missing data in products_df === #

print("Basic missing data check:\n")
print(products_df.isnull().sum())


print("\nPercentage of missing data:\n")
print((products_df.isnull().mean() * 100).round(2))

# === Detailed look at missing Category === #
missing_category = products_df[products_df["Category"].isnull()]
print("\nSample rows with missing Category:\n")
print(missing_category.head())

# === Check if missing Category correlates with Price === #
print("\nPrice stats for missing Category vs not missing:\n")
print(products_df.groupby(products_df["Category"].isnull())["Price"].describe())

# === Detailed look at missing Description === #
missing_description = products_df[products_df["Description"].isnull()]
print("\nSample rows with missing Description:\n")
print(missing_description.head())

# === Check if missing Description correlates with Category === #
print("\nCount of missing Descriptions by Category:\n")
print(products_df.groupby("Category")["Description"].apply(lambda x: x.isnull().sum()))


Basic missing data check:

ProductID       0
ProductName     0
Category        5
Price           0
Description    12
dtype: int64

Percentage of missing data:

ProductID       0.0
ProductName     0.0
Category        5.0
Price           0.0
Description    12.0
dtype: float64

Sample rows with missing Category:

   ProductID ProductName Category   Price                 Description
1    PRD0001   Product 1     None  320.02                        None
11   PRD0011  Product 11     None   84.80  Description for product 11
34   PRD0034  Product 34     None  471.74  Description for product 34
50   PRD0050  Product 50     None  454.59  Description for product 50
69   PRD0069  Product 69     None  297.49  Description for product 69

Price stats for missing Category vs not missing:

          count        mean         std    min      25%     50%     75%  \
Category                                                                  
False      95.0  236.939684  146.153685   4.22  123.105  208.14  36

In [ ]:
# === Investigate missing data in customers_df === #

print("Basic missing data check:\n")
print(customers_df.isnull().sum())

print("\nPercentage of missing data:\n")
print((customers_df.isnull().mean() * 100).round(2))

# === Detailed look at missing Address === #
missing_address = customers_df[customers_df["Address"].isnull()]
print("\nSample rows with missing Address:\n")
print(missing_address.head())

# === Check duplicate CustomerIDs === #
duplicate_counts = customers_df["CustomerID"].value_counts()
duplicates = duplicate_counts[duplicate_counts > 1]
print("\nDuplicate CustomerIDs:\n")
print(duplicates)

print(f"\nTotal duplicate CustomerIDs: {duplicates.shape[0]}")

# === Check for bad email formats === #
print("\nChecking for likely bad email formats:\n")
bad_email_mask = ~customers_df["Email"].str.contains("@") | ~customers_df["Email"].str.contains(".")
bad_emails = customers_df[bad_email_mask]
print(bad_emails.head())

print(f"\nTotal suspected bad emails: {bad_emails.shape[0]}")


Basic missing data check:

CustomerID     0
Name           0
Email          0
Address       26
dtype: int64

Percentage of missing data:

CustomerID     0.0
Name           0.0
Email          0.0
Address       13.0
dtype: float64

Sample rows with missing Address:

   CustomerID         Name                   Email Address
9    CUST0012   Customer 9   customer9@example.com    None
15   CUST0144  Customer 15            bad_email_15    None
21   CUST0182  Customer 21            bad_email_21    None
25   CUST0061  Customer 25  customer25@example.com    None
31   CUST0057  Customer 31  customer31@example.com    None

Duplicate CustomerIDs:

CustomerID
CUST0068    5
CUST0098    4
CUST0046    4
CUST0055    4
CUST0139    4
CUST0143    4
CUST0084    4
CUST0151    4
CUST0188    3
CUST0022    3
CUST0095    3
CUST0146    3
CUST0057    3
CUST0116    3
CUST0089    3
CUST0099    3
CUST0100    3
CUST0140    3
CUST0011    3
CUST0061    2
CUST0018    2
CUST0182    2
CUST0051    2
CUST0028    2
CUST0152 

In [ ]:
# === Investigate missing data in orders_df === #
print("Basic missing data check:\n")
print(orders_df.isnull().sum())

print("\nPercentage of missing data:\n")
print((orders_df.isnull().mean() * 100).round(2))

# === Detailed look at missing CustomerID (guest checkouts) === #
guest_orders = orders_df[orders_df["CustomerID"].isnull()]
print("\nSample guest checkout orders (missing CustomerID):\n")
print(guest_orders.head())

print(f"\nTotal guest checkout orders: {guest_orders.shape[0]}")

# === Check for inconsistent Status values (case issues, typos) === #
print("\nUnique order Status values:\n")
print(orders_df["Status"].unique())

# Suggest standardizing Status
orders_df["Status_clean"] = orders_df["Status"].str.strip().str.lower()
print("\nSample standardized Status values:\n")
print(orders_df["Status_clean"].value_counts())

# === Inspect date formats === #
print("\nSample of OrderDate formats:\n")
print(orders_df["OrderDate"].sample(10))

# Attempt parsing dates to see failures
def try_parse_date(date_str):
    for fmt in ("%Y-%m-%d", "%d/%m/%Y"):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return None

orders_df["ParsedDate"] = orders_df["OrderDate"].apply(try_parse_date)
invalid_dates = orders_df[orders_df["ParsedDate"].isnull()]

print("\nOrders with unparseable dates:\n")
print(invalid_dates.head())

print(f"\nTotal unparseable dates: {invalid_dates.shape[0]}")


Basic missing data check:

OrderID         0
CustomerID    113
ProductID       0
OrderDate       0
Quantity        0
Status          0
dtype: int64

Percentage of missing data:

OrderID        0.0
CustomerID    22.6
ProductID      0.0
OrderDate      0.0
Quantity       0.0
Status         0.0
dtype: float64

Sample guest checkout orders (missing CustomerID):

     OrderID CustomerID ProductID   OrderDate  Quantity     Status
1   ORD00001       None   PRD0072  20/01/2022         1       PAID
9   ORD00009       None   PRD0060  17/07/2023         1  Cancelled
16  ORD00016       None   PRD0087  2023-03-25         2       PAID
17  ORD00017       None   PRD0010  08/04/2023         2    Pending
32  ORD00032       None   PRD0097  2022-12-29         1    Pending

Total guest checkout orders: 113

Unique order Status values:

['Shipped' 'PAID' 'pending' 'Refunded' 'Cancelled' 'Paid' 'Pending']

Sample standardized Status values:

Status_clean
pending      157
paid         139
cancelled     76
refu

In [ ]:
# ==========================================
# CROSS-TABLE DATA INTEGRITY CHECKS
# ==========================================

print("\n=== 1️⃣ Check: Missing CustomerIDs in Orders (Guest Checkouts) ===")
guest_checkout_count = orders_df["CustomerID"].isnull().sum()
total_order_count = len(orders_df)
guest_pct = 100 * guest_checkout_count / total_order_count
print(f"Guest checkouts (missing CustomerID): {guest_checkout_count} / {total_order_count} ({guest_pct:.2f}%)\n")

# Sample of guest orders
print("Sample guest checkout orders:")
print(orders_df[orders_df["CustomerID"].isnull()].head())

# ------------------------------------------

print("\n=== 2️⃣ Check: Orders with ProductIDs NOT in Products Table ===")
missing_products_mask = ~orders_df["ProductID"].isin(products_df["ProductID"])
missing_products_count = missing_products_mask.sum()
print(f"Orders referencing missing ProductIDs: {missing_products_count}")

if missing_products_count > 0:
    print("\nSample of problematic orders:")
    print(orders_df[missing_products_mask].head())

# ------------------------------------------

print("\n=== 3️⃣ Check: Orders with CustomerIDs NOT in Customers Table ===")
known_customer_orders = orders_df["CustomerID"].notnull()
invalid_customers_mask = known_customer_orders & ~orders_df["CustomerID"].isin(customers_df["CustomerID"])
invalid_customers_count = invalid_customers_mask.sum()
print(f"Orders with CustomerIDs not found in Customers Table: {invalid_customers_count}")

if invalid_customers_count > 0:
    print("\nSample of problematic orders:")
    print(orders_df[invalid_customers_mask].head())

# ------------------------------------------

print("\n=== 4️⃣ Check: Duplicate CustomerIDs in Customers Table ===")
customer_id_counts = customers_df["CustomerID"].value_counts()
duplicate_customers = customer_id_counts[customer_id_counts > 1]

print(f"Number of CustomerIDs with duplicates: {duplicate_customers.shape[0]}")

if not duplicate_customers.empty:
    print("\nSample of duplicated CustomerIDs and their counts:")
    print(duplicate_customers.head())

    # Show sample duplicated records
    duplicated_records = customers_df[customers_df["CustomerID"].isin(duplicate_customers.index)]
    print("\nSample duplicated customer records:")
    print(duplicated_records.head())



=== 1️⃣ Check: Missing CustomerIDs in Orders (Guest Checkouts) ===
Guest checkouts (missing CustomerID): 113 / 500 (22.60%)

Sample guest checkout orders:
     OrderID CustomerID ProductID   OrderDate  Quantity     Status
1   ORD00001       None   PRD0072  20/01/2022         1       PAID
9   ORD00009       None   PRD0060  17/07/2023         1  Cancelled
16  ORD00016       None   PRD0087  2023-03-25         2       PAID
17  ORD00017       None   PRD0010  08/04/2023         2    Pending
32  ORD00032       None   PRD0097  2022-12-29         1    Pending

=== 2️⃣ Check: Orders with ProductIDs NOT in Products Table ===
Orders referencing missing ProductIDs: 0

=== 3️⃣ Check: Orders with CustomerIDs NOT in Customers Table ===
Orders with CustomerIDs not found in Customers Table: 140

Sample of problematic orders:
     OrderID CustomerID ProductID   OrderDate  Quantity    Status
3   ORD00003   CUST0056   PRD0048  2022-11-11         3  Refunded
6   ORD00006   CUST0168   PRD0011  2023-05-13   